## Week 4 - Little Lemon analysis and sales report

You have gained all the necessary skills and knowledge required to interact with the back-end MySQL database from your Python-based front-end application via the standard database client MySQL Connector/Python. 

Follow the instructions in the exercises given below and create a Jupyter notebook to produce a Little Lemon analysis and sales report. You need to submit this report for your peers to review.

## Task 1:
Complete the following steps to establish a connection pool:

To create a connection pool, import MySQLConnectionPool class from MySQL Connector/Python.

To find the information on the error, import the Error class from MySQL Connector/Python.

Define your database configurations as a Python dictionary object called dbconfig.

Establish a connection pool [pool_name = pool_b] with two connections. 

Implement error handling using a try-except block in case the connection fails. 

In [1]:
from mysql.connector.pooling import MySQLConnectionPool
from mysql.connector import Error

dbconfig = {
    "database":"little_lemon",
    "user" : "root",
    "password" : "root"
}

try:
    pool = MySQLConnectionPool(pool_name = "pool_b",
                           pool_size = 2, #default is 5
                           **dbconfig)
    print("The connection pool is created with a name: ",pool.pool_name)
    print("The pool size is:",pool.pool_size)

except Error as er:
    print("Error code:", er.errno)
    print("Error message:", er.msg)
    
print("Getting a connection from the pool.")
connection = pool.get_connection()

# Create cursor object to communicate with entire MySQL database
print("Creating a cursor object.")
cursor = connection.cursor()


The connection pool is created with a name:  pool_b
The pool size is: 2
Getting a connection from the pool.
Creating a cursor object.


## Task 2:
Three guests are trying to book dinner slots simultaneously. Get the connections from pool_b and insert the following data in the Bookings table:
TIP: You need to add a connection to connect the third guest.

Guest 1:
Table Number: 8
First Name: Anees
Last Name: Java
Booking Time: 18:00
EmployeeID: 6

Guest 2:
Table Number: 5
First Name: Bald
Last Name: Vin
Booking Time: 19:00
EmployeeID: 6


Guest 3:
Table Number: 12
First Name: Jay
Last Name: Kon
Booking Time: 19:30 
EmployeeID: 6

Return all the connections back to the pool. 

TIP: The pool size is two. However, you have three connected users. You can only return two connections. Returning a third connection will raise a PoolError. Use try-except to print the error message.  

In [5]:
for connection in connection_list:
    try:
        connection.close()
    except Error as err:
        print("\nConnection can't be returned to the pool")
        print("Error message", err.msg)
        print("Error code", err.errno)


Connection can't be returned to the pool
Error message Failed adding connection; queue is full
Error code -1

Connection can't be returned to the pool
Error message Failed adding connection; queue is full
Error code -1

Connection can't be returned to the pool
Error message Failed adding connection; queue is full
Error code -1


In [6]:
import mysql.connector as connector

insert_bookings="""
INSERT INTO Bookings (TableNo, GuestFirstName, 
GuestLastName, BookingSlot, EmployeeID)
VALUES
"""
connection_list = []
value_list = [(8,'Anna','Java','18:00:00',6), (5,'Bald','Vin','19:00:00',6), (12,'Jay','Kon','19:30:00',6)]
for value in value_list:
    guest = value[1]
    try:      
        guest_connected = pool.get_connection()
        print("[{}] is connected.\n".format(guest))
        print("Creating a cursor object.")
        cursor = guest_connected.cursor()
        connection_list.append(guest_connected)
    except:
        print("No more connections are available.")
        print("Adding new connection in the pool.")
        connection=connector.connect(user="root",password="root")
        
        # Add the connection into the pool
        pool.add_connection(cnx=connection)
        print("A new connection is added in the pool.\n")       
        user_connected = pool.get_connection()
        connection_list.append(user_connected)
        print("[{}] is connected.\n".format(guest))  
        cursor = user_connected.cursor()
    finally:
        str_insertion = insert_bookings + str(value)
        cursor.execute(str_insertion)
        print("Finished inserting", guest)

for connection in connection_list:
    try:
        connection.close()
    except Error as err:
        print("\nConnection can't be returned to the pool")
        print("Error message", err.msg)
        print("Error code", err.errno)


[Anna] is connected.

Creating a cursor object.
Finished inserting Anna
[Bald] is connected.

Creating a cursor object.
Finished inserting Bald
No more connections are available.
Adding new connection in the pool.
A new connection is added in the pool.

[Jay] is connected.

Finished inserting Jay

Connection can't be returned to the pool
Error message Failed adding connection; queue is full
Error code -1


## Task 3: 

Create a report containing the following information:

The name and EmployeeID of the Little Lemon manager.

The name and role of the employee who receives the highest salary.

The number of guests booked between 18:00 and 20:00.

The full name and BookingID of all guests waiting to be seated with the receptionist in sorted order with respect to their BookingSlot.

In [7]:
import mysql.connector as connector

def query_show(cursor, select_query):
    cursor.execute(select_query)
    results = cursor.fetchall()
    columns = cursor.column_names
    print(columns)
    for result in results:
        print(result)
    return
    
try:      
    guest_connected = pool.get_connection()
    cursor = guest_connected.cursor()
except:
    print("No more connections are available.")
    print("Adding new connection in the pool.")
    connection = connector.connect(user="root",password="root")
    
    # Add the connection into the pool
    pool.add_connection(cnx=connection)
    user_connected = pool.get_connection()
    cursor = user_connected.cursor()
            
select_query="""
SELECT Name, EmployeeID FROM Employees WHERE Role = "Manager";
"""

print("\nThe EmployeeID and name of the Little Lemon manager")
query_show(cursor, select_query)

select_query="""
SELECT 
Name, EmployeeID FROM Employees 
ORDER BY Annual_Salary DESC LIMIT 1;
"""
print("\nThe name and ID of the employee who receives the highest salary.")
query_show(cursor, select_query)

select_query="""
SELECT COUNT(BookingID) 
FROM Bookings 
WHERE BookingSlot BETWEEN '18:00:00' AND '20:00:00';
"""
print("\nThe number of guests booked between 18:00 and 20:00")
query_show(cursor, select_query)

select_query="""
SELECT Bookings.BookingID AS ID,  
CONCAT(GuestFirstName,' ',GuestLastName) AS GuestName, 
Role AS Employee
FROM Bookings LEFT JOIN Employees 
ON Employees.EmployeeID = Bookings.EmployeeID
WHERE Employees.Role = "Receptionist"
ORDER BY BookingSlot DESC;
"""
print("\nThe following guests are waiting to be seated:")
query_show(cursor, select_query)



The EmployeeID and name of the Little Lemon manager
('Name', 'EmployeeID')
('Mario Gollini', 1)

The name and ID of the employee who receives the highest salary.
('Name', 'EmployeeID')
('Mario Gollini', 1)

The number of guests booked between 18:00 and 20:00
('COUNT(BookingID)',)
(4,)

The following guests are waiting to be seated:
('ID', 'GuestName', 'Employee')


## Task 4: 
Create a stored procedure named BasicSalesReport that returns the following statistics: 

Total sales

Average sale

Minimum bill paid

Maximum bill paid

In [9]:
stored_procedure_query = """
CREATE PROCEDURE BasicSalesReport()

BEGIN

SELECT SUM(BillAmount) AS Total, 
AVG(BillAmount) AS Average, 
MIN(BillAmount) AS Minimum, 
MAX(BillAmount) AS Maximum 
FROM Orders;

END

"""

# Execute the query
cursor.execute(stored_procedure_query)

#********************************************#

# Call the stored procedure with its name
cursor.callproc("BasicSalesReport")

# Retrieve recrods in "dataset"
results = next( cursor.stored_results() )
dataset = results.fetchall()

# Retrieve column names using list comprehension in a 'for' loop 
for column_id in cursor.stored_results():
    cols = [ column[0] for column in column_id.description ]

# Print column names

# Print data 
for data in dataset:
    print("\t",cols[0],":",data[0])
    print("\t",cols[1],":",data[1])
    print("\t",cols[2],":",data[2])
    print("\t",cols[3],":",data[3])

	 Total : 243
	 Average : 48.6000
	 Minimum : 37
	 Maximum : 86


## Task 5:
Little Lemon needs to display the next three upcoming bookings from the Bookings table on the kitchen screen to notify their chefs which orders are due next. To complete this task, carry out the following steps:

Get a connection from the pool.

Create a buffered cursor.

Combine the data from the Bookings and the Employee tables. Sort the retrieved records in ascending order. Then display the information of the first three guests. 

Returned the connection back to the pool.


The output should be as follows:

[BookingSlot]

[Guest_name]

[Assigned to: Employee Name [Employee Role]]

In [10]:
try:      
    guest_connected = pool.get_connection()
    cursor = guest_connected.cursor()
except:
    print("No more connections are available.")
    print("Adding new connection in the pool.")
    connection = connector.connect(user="root",password="root")
    
    # Add the connection into the pool
    pool.add_connection(cnx=connection)
    user_connected = pool.get_connection()
    cursor = user_connected.cursor()
            
select_query="""
SELECT BookingSlot, CONCAT(GuestFirstName, GuestLastName) AS GuestName, Name AS EmployeeName, Role 
FROM Bookings INNER JOIN Employees 
ON Bookings.EmployeeID = Employees.EmployeeID 
ORDER BY BookingSlot ASC 
LIMIT 3;
"""

print("\nThree upcoming bookings from the Bookings table")
cursor.execute(select_query)
results = cursor.fetchall()

for result in results:
    print("[",result[0],"]", "[", result[1], "]", "[Assigned to: ", result[2], "[", result[3],"]]")


The name and EmployeeID of the Little Lemon manager
[ 15:00:00 ] [ VanessaMcCarthy ] [Assigned to:  Giorgos Dioudis [ Head Chef ]]
[ 17:30:00 ] [ MarcosRomero ] [Assigned to:  Fatma Kaya [ Assistant Chef ]]
[ 18:30:00 ] [ HirokiYamane ] [Assigned to:  Adrian Gollini [ Assistant Manager ]]


In [11]:
# Let's close the cursor and the connection
try:
    if guest_connected.is_connected():
        cursor.close()
        print("The cursor is closed.")
        guest_connected.close()
        print("MySQL connection is closed.")
    else:
        print("Connection is already closed")
except Error as err:
    print("Error message is", err.msg)
    print("Error code is", err.errno)
    if user_connected.is_connected():
        cursor.close()
        print("The cursor is closed.")
        user_connected.close()
        print("MySQL connection is closed.")
    else:
        print("Connection is already closed")   

The cursor is closed.
MySQL connection is closed.
